In [1]:
import tensorflow as tf
import pandas as pd

tf.test.is_built_with_cuda()

2023-10-03 14:21:20.494005: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-03 14:21:22.377057: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


True

In [2]:
dataset = pd.read_csv(r'Dataset.csv')
dataset['Date'] = pd.to_datetime(dataset['Date'])
# Setting date column to index
# dataset = dataset.set_index(['Date'], drop=False)

In [3]:
dataset.drop(dataset.columns[[2, 3, 4, 5, 6]], axis=1, inplace=True)

In [4]:
dataset = dataset.sort_values(by=["Date"], ascending=True)

In [5]:
dataset['Date'][0]

Timestamp('2023-10-02 00:00:00')

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [7]:
temp = dataset.drop(["Price"], axis=1)
print(temp.shape)
temp

(1279, 1)


,Date
1278,2018-10-02
1277,2018-10-03
1276,2018-10-04
1275,2018-10-05
1274,2018-10-08
...,...
4,2023-09-27
3,2023-09-28
2,2023-09-29
1,2023-10-01


In [8]:
# Data normalization
x = StandardScaler().fit_transform(dataset.drop(["Price"], axis=1))
y = dataset['Price'].values

x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.1,
    shuffle=False
)

In [14]:
from keras.layers import GRU, Dense
from keras.models import Sequential, load_model
import numpy as np

In [10]:
model = Sequential()
model.add(GRU(7, input_shape=(x_train.shape[1], 1), activation='linear', kernel_initializer='lecun_uniform', return_sequences=False))
model.add(Dense(1))
model.compile(optimizer="adam", loss=tf.keras.metrics.mean_squared_error,
              metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
model.summary()

2023-10-03 14:21:43.858362: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


2023-10-03 14:21:44.011940: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-03 14:21:44.012076: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-03 14:21:44.015819: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-03 14:21:44.015899: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-03 14:21:44.015932: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 7)                 210       
                                                                 
 dense (Dense)               (None, 1)                 8         
                                                                 
Total params: 218 (872.00 Byte)
Trainable params: 218 (872.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
model.fit(x_train[:, :, np.newaxis], y_train, epochs=100, batch_size=1, verbose=1, shuffle=False)

Epoch 1/100
1151/1151 [==============================] - 5s 3ms/step - loss: 391.0580 - rmse: 19.7752
Epoch 2/100
1151/1151 [==============================] - 3s 3ms/step - loss: 8.4729 - rmse: 2.9108
Epoch 3/100
1151/1151 [==============================] - 3s 2ms/step - loss: 3.3602 - rmse: 1.8331
Epoch 4/100
1151/1151 [==============================] - 2s 2ms/step - loss: 2.7609 - rmse: 1.6616
Epoch 5/100
1151/1151 [==============================] - 2s 1ms/step - loss: 2.6139 - rmse: 1.6167
Epoch 6/100
1151/1151 [==============================] - 2s 1ms/step - loss: 1.9082 - rmse: 1.3814
Epoch 7/100
1151/1151 [==============================] - 2s 1ms/step - loss: 1.4110 - rmse: 1.1879
Epoch 8/100
1151/1151 [==============================] - 2s 1ms/step - loss: 1.1291 - rmse: 1.0626
Epoch 9/100
1151/1151 [==============================] - 2s 1ms/step - loss: 0.9734 - rmse: 0.9866
Epoch 10/100
1151/1151 [==============================] - 2s 1ms/step - loss: 0.8850 - rmse: 0.9407
Epoch 

In [42]:
y_pred = model.predict(x_test[:, :, np.newaxis])

4/4 [==============================] - 0s 3ms/step


In [43]:
print(y_pred)

[[36.958412]
 [36.975574]
 [36.981228]
 [36.98685 ]
 [36.992447]
 [36.99801 ]
 [37.01451 ]
 [37.019955]
 [37.025364]
 [37.03075 ]
 [37.036102]
 [37.051983]
 [37.057224]
 [37.06243 ]
 [37.067608]
 [37.072765]
 [37.088062]
 [37.0931  ]
 [37.09812 ]
 [37.103107]
 [37.108067]
 [37.122803]
 [37.12766 ]
 [37.132492]
 [37.137302]
 [37.142082]
 [37.156277]
 [37.16096 ]
 [37.165623]
 [37.170254]
 [37.174866]
 [37.188553]
 [37.19307 ]
 [37.197567]
 [37.20203 ]
 [37.20648 ]
 [37.21968 ]
 [37.224037]
 [37.22837 ]
 [37.23269 ]
 [37.236977]
 [37.24972 ]
 [37.253933]
 [37.258118]
 [37.26228 ]
 [37.266426]
 [37.278732]
 [37.282806]
 [37.286842]
 [37.290863]
 [37.29487 ]
 [37.306767]
 [37.310696]
 [37.314606]
 [37.318497]
 [37.322365]
 [37.333874]
 [37.33767 ]
 [37.34145 ]
 [37.34521 ]
 [37.348957]
 [37.360092]
 [37.363766]
 [37.36743 ]
 [37.371067]
 [37.374695]
 [37.385475]
 [37.389038]
 [37.392586]
 [37.39611 ]
 [37.39962 ]
 [37.41007 ]
 [37.413525]
 [37.416958]
 [37.420376]
 [37.42378 ]
 [37.433914]

In [44]:
model.save('model.h5')

/home/flight/miniconda3/envs/tf/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
from datetime import datetime, timedelta

In [12]:
start = '2023-10-3'
end = '2023-10-23'

In [13]:
def forecast(model, start: str, end: str):

    data = {"Date": []}

    start_y, start_m, start_d = start.split('-')
    end_y, end_m, end_d = end.split('-')

    date = datetime(int(start_y), int(start_m), int(start_d), 0, 0, 0)
    stop = datetime(int(end_y), int(end_m), int(end_d), 0, 0, 0)

    for i in range(30):
        data['Date'].append(pd.Timestamp(date))
        if date == stop:
            break
        date += timedelta(days=1)
    
    x_new = pd.DataFrame(data)

    return x_new

    

In [15]:
new_model = load_model(r'model.h5')

In [16]:
x_new = StandardScaler().fit_transform(forecast(1, start, end))
pred = new_model.predict(x_new[:, :, np.newaxis])

1/1 [==============================] - 2s 2s/step


In [28]:
print(len(pred))
print(pred[20][0])
pred

21
37.535374


array([[26.898527],
       [26.451122],
       [26.180662],
       [26.145796],
       [26.373257],
       [26.795553],
       [27.231945],
       [27.477673],
       [27.426933],
       [27.115517],
       [26.702883],
       [26.468082],
       [26.806974],
       [28.083475],
       [30.239798],
       [32.63644 ],
       [34.600792],
       [35.923   ],
       [36.732006],
       [37.22156 ],
       [37.535374]], dtype=float32)

In [29]:
max(pred)

array([37.535374], dtype=float32)